<a href="https://colab.research.google.com/github/viet4777/UsefulTools/blob/main/Tool_th%C6%B0%E1%BB%9Dng_d%C3%B9ng_(2026).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Useful Tools
import re
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd
import math

# Dictionary of atomic masses (g/mol) - common elements
ATOMIC_MASSES = {
    'H': 1.00794, 'He': 4.002602, 'Li': 6.941, 'Be': 9.012182, 'B': 10.811,
    'C': 12.0107, 'N': 14.0067, 'O': 15.9994, 'F': 18.9984032, 'Ne': 20.1797,
    'Na': 22.989769, 'Mg': 24.3050, 'Al': 26.9815386, 'Si': 28.0855, 'P': 30.973762,
    'S': 32.065, 'Cl': 35.453, 'Ar': 39.948, 'K': 39.0983, 'Ca': 40.078,
    'Sc': 44.955912, 'Ti': 47.867, 'V': 50.9415, 'Cr': 51.9961, 'Mn': 54.938045,
    'Fe': 55.845, 'Co': 58.933195, 'Ni': 58.6934, 'Cu': 63.546, 'Zn': 65.38,
    'Ga': 69.723, 'Ge': 72.64, 'As': 74.92160, 'Se': 78.96, 'Br': 79.904,
    'Kr': 83.798, 'Rb': 85.4678, 'Sr': 87.62, 'Y': 88.90585, 'Zr': 91.224,
    'Nb': 92.90638, 'Mo': 95.96, 'Tc': 98.0, 'Ru': 101.07, 'Rh': 102.90550,
    'Pd': 106.42, 'Ag': 107.8682, 'Cd': 112.411, 'In': 114.818, 'Sn': 118.710,
    'Sb': 121.760, 'Te': 127.60, 'I': 126.90447, 'Xe': 131.293, 'Cs': 132.9054519,
    'Ba': 137.327, 'La': 138.90547, 'Ce': 140.116, 'Pr': 140.90765, 'Nd': 144.242,
    'Pm': 145.0, 'Sm': 150.36, 'Eu': 151.964, 'Gd': 157.25, 'Tb': 158.92535,
    'Dy': 162.500, 'Ho': 164.93032, 'Er': 167.259, 'Tm': 168.93421, 'Yb': 173.054,
    'Lu': 174.9668, 'Hf': 178.49, 'Ta': 180.94788, 'W': 183.84, 'Re': 186.207,
    'Os': 190.23, 'Ir': 192.217, 'Pt': 195.084, 'Au': 196.966569, 'Hg': 200.59,
    'Tl': 204.3833, 'Pb': 207.2, 'Bi': 208.98040, 'Po': 209.0, 'At': 210.0,
    'Rn': 222.0, 'Fr': 223.0, 'Ra': 226.0, 'Ac': 227.0, 'Th': 232.03806,
    'Pa': 231.03588, 'U': 238.02891, 'Np': 237.0, 'Pu': 244.0, 'Am': 243.0,
    'Cm': 247.0, 'Bk': 247.0, 'Cf': 251.0, 'Es': 252.0, 'Fm': 257.0,
    'Md': 258.0, 'No': 259.0, 'Lr': 262.0, 'Rf': 267.0, 'Db': 268.0,
    'Sg': 271.0, 'Bh': 272.0, 'Hs': 270.0, 'Mt': 276.0, 'Ds': 281.0,
    'Rg': 280.0, 'Cn': 285.0, 'Nh': 284.0, 'Fl': 289.0, 'Mc': 288.0,
    'Lv': 293.0, 'Ts': 294.0, 'Og': 294.0
}

# Mapping for subscript numbers (0-9)
SUBSCRIPT = {
    '0': '₀', '1': '₁', '2': '₂', '3': '₃', '4': '₄',
    '5': '₅', '6': '₆', '7': '₇', '8': '₈', '9': '₉'
}

# Mapping for superscript numbers (0-9) for atomic percentages
SUPERSCRIPT = {
    '0': '⁰', '1': '¹', '2': '²', '3': '³', '4': '⁴',
    '5': '⁵', '6': '⁶', '7': '⁷', '8': '⁸', '9': '⁹'
}

def parse_chemical_formula(formula):
    """
    Parse chemical formula like Al82Fe18 into elements and atomic percentages
    If no percentages are given (e.g., AlNiCoTiFeMo), divide 100% equally among elements

    Args:
        formula (str): Chemical formula like Al82Fe18, Cu50Zr50, Al80Fe10Ni10, or AlNiCoTiFeMo

    Returns:
        list: List of tuples [(element, percentage), ...]
    """
    # First, check if formula contains any numbers
    if not any(char.isdigit() for char in formula):
        # No numbers found, extract all chemical elements and divide equally
        pattern = r'([A-Z][a-z]?)'
        elements = re.findall(pattern, formula)

        if not elements:
            return []

        # Calculate equal percentage for each element
        percentage_per_element = 100.0 / len(elements)

        return [(element, percentage_per_element) for element in elements]

    else:
        # Formula contains numbers, parse normally
        # Regex to find element symbols (capital letter followed by optional lowercase) and numbers
        pattern = r'([A-Z][a-z]?)(\d*\.?\d*)'
        matches = re.findall(pattern, formula)

        elements = []
        for element, percentage in matches:
            if percentage:
                # Convert percentage to float
                try:
                    perc_float = float(percentage)
                    elements.append((element, perc_float))
                except ValueError:
                    # If conversion fails, skip this element
                    continue
            else:
                # If no percentage specified, assume equal share with other elements without percentages
                # We'll handle this case separately
                elements.append((element, None))

        # Check if any element has None percentage
        none_indices = [i for i, (_, perc) in enumerate(elements) if perc is None]
        if none_indices:
            # Calculate remaining percentage for elements without specified percentages
            specified_total = sum(perc for _, perc in elements if perc is not None)
            remaining_percentage = 100.0 - specified_total
            if remaining_percentage < 0:
                # If total exceeds 100%, normalize all
                total = sum(perc for _, perc in elements if perc is not None)
                for i, (elem, perc) in enumerate(elements):
                    if perc is not None:
                        elements[i] = (elem, (perc / total) * 100)
                    else:
                        elements[i] = (elem, 0)
            else:
                # Distribute remaining percentage equally among elements without specified percentages
                equal_share = remaining_percentage / len(none_indices)
                for i in none_indices:
                    elements[i] = (elements[i][0], equal_share)

        return elements

def convert_to_subscript(text):
    """Convert regular numbers to subscript numbers"""
    result = []
    for char in text:
        if char in SUBSCRIPT:
            result.append(SUBSCRIPT[char])
        else:
            result.append(char)
    return ''.join(result)

def convert_to_superscript(text):
    """Convert regular numbers to superscript numbers"""
    result = []
    for char in text:
        if char in SUPERSCRIPT:
            result.append(SUPERSCRIPT[char])
        else:
            result.append(char)
    return ''.join(result)

def calculate_element_masses(elements):
    """
    Calculate masses of each element based on atomic percentages

    Args:
        elements (list): List of tuples [(element, percentage), ...]

    Returns:
        dict: Dictionary with element masses and total mass
    """
    # Normalize percentages to sum to 100%
    total_percentage = sum(perc for _, perc in elements)
    if total_percentage == 0:
        return {}

    normalized_elements = []
    for element, percentage in elements:
        normalized_percentage = (percentage / total_percentage) * 100
        normalized_elements.append((element, normalized_percentage))

    # Calculate the sum of (atomic % × atomic mass) for normalization
    sum_atomic_percent_mass = 0
    element_data_pre = []

    for element, percentage in normalized_elements:
        if element in ATOMIC_MASSES:
            atomic_mass = ATOMIC_MASSES[element]
            sum_atomic_percent_mass += percentage * atomic_mass
            element_data_pre.append({
                'element': element,
                'percentage': percentage,
                'atomic_mass': atomic_mass
            })
        else:
            element_data_pre.append({
                'element': element,
                'percentage': percentage,
                'atomic_mass': 'Unknown'
            })

    results = []
    total_mass_100g = 0

    for i, data in enumerate(element_data_pre, start=1):
        element = data['element']
        percentage = data['percentage']
        atomic_mass = data['atomic_mass']

        if atomic_mass != 'Unknown' and sum_atomic_percent_mass > 0:
            # Calculate mass in 100g of compound (correct formula)
            mass_in_100g = (percentage * atomic_mass / sum_atomic_percent_mass) * 100
            total_mass_100g += mass_in_100g

            # Calculate mass percentage
            mass_percent = (percentage * atomic_mass / sum_atomic_percent_mass) * 100

            # Calculate mass in 10g
            mass_in_10g = mass_percent / 10.0

            results.append({
                'No.': i,
                'Element': element,
                'Atomic %': percentage,
                'Atomic Mass (g/mol)': atomic_mass,
                'Mass %': mass_percent,
                'Mass in 10g (g)': mass_in_10g
            })
        else:
            results.append({
                'No.': i,
                'Element': element,
                'Atomic %': percentage,
                'Atomic Mass (g/mol)': 'Unknown',
                'Mass %': 'Unknown',
                'Mass in 10g (g)': 'Unknown'
            })

    return {
        'element_data': results,
        'total_mass': total_mass_100g,
        'normalized_elements': normalized_elements
    }

def create_formula_with_subscript(elements):
    """Create chemical formula with subscript numbers"""
    formula_parts = []
    for element, percentage in elements:
        # Format percentage to 2 decimal places
        if percentage.is_integer():
            perc_str = str(int(percentage))
        else:
            perc_str = f"{percentage:.2f}"

        if len(elements) == 1 and abs(percentage - 100.0) < 0.01:
            # Single element, no subscript needed
            formula_parts.append(element)
        else:
            formula_parts.append(f"{element}{convert_to_subscript(perc_str)}")

    return ''.join(formula_parts)

def create_formula_with_percentage_superscript(elements):
    """Create chemical formula with atomic percentages as superscript"""
    formula_parts = []
    for element, percentage in elements:
        # Format percentage to 2 decimal places
        if percentage.is_integer():
            perc_str = str(int(percentage))
        else:
            perc_str = f"{percentage:.2f}"
        formula_parts.append(f"{element}{convert_to_superscript(perc_str)}")

    return ''.join(formula_parts)

# Tab 2: Force to Stress Converter Functions
def calculate_stress_from_force(force_kn, diameter_mm):
    """
    Calculate stress (MPa) from force (kN) and diameter (mm)

    Formula: Stress (MPa) = Force (kN) * 1000 / Area (mm²)
    Area = π * (diameter/2)²
    """
    # Calculate cross-sectional area in mm²
    radius_mm = diameter_mm / 2.0
    area_mm2 = math.pi * (radius_mm ** 2)

    # Convert force from kN to N (1 kN = 1000 N)
    force_n = force_kn * 1000

    # Calculate stress in MPa (N/mm²)
    stress_mpa = force_n / area_mm2 if area_mm2 > 0 else 0

    return stress_mpa, area_mm2

def calculate_force_from_stress(stress_mpa, diameter_mm):
    """
    Calculate force (kN) from stress (MPa) and diameter (mm)

    Formula: Force (kN) = Stress (MPa) * Area (mm²) / 1000
    Area = π * (diameter/2)²
    """
    # Calculate cross-sectional area in mm²
    radius_mm = diameter_mm / 2.0
    area_mm2 = math.pi * (radius_mm ** 2)

    # Calculate force in N, then convert to kN
    force_n = stress_mpa * area_mm2
    force_kn = force_n / 1000

    return force_kn, area_mm2

# Create UI widgets for Tab 1
input_formula = widgets.Text(
    value='AlNiCoTiFeMo',
    placeholder='Enter formula like Al82Fe18, Cu50Zr50, or AlNiCoTiFeMo (equal distribution)',
    description='Formula:',
    disabled=False,
    layout=widgets.Layout(width='400px')
)

calculate_button = widgets.Button(
    description='Calculate',
    button_style='primary',
    icon='calculator',
    layout=widgets.Layout(width='150px')
)

clear_button = widgets.Button(
    description='Clear',
    button_style='warning',
    icon='eraser',
    layout=widgets.Layout(width='150px')
)

# Output areas for Tab 1
result_output = widgets.Output()
formula_display = widgets.HTML(value='')
mass_table_output = widgets.Output()
summary_output = widgets.Output()

# Create example buttons for Tab 1
example_formulas = [
    'Al82Fe18',
    'Al82Fe12Nb2Ni2Ti2',
    'Al20(NiTiNbCo)80',  #al20ni20ti20nb20co20
    'Ti50Cu25Ni25',
    'Mg65Cu25Y10',
    'AlNiCoTiFeMo',  # Equal distribution example
    'CuNbTi',        # Equal distribution example
    'FeNiCoCr'       # Equal distribution example
]

example_buttons = []
for formula in example_formulas:
    btn = widgets.Button(
        description=formula,
        button_style='info',
        layout=widgets.Layout(width='120px', margin='0 5px 0 0')
    )
    btn.formula = formula  # Store formula as attribute
    example_buttons.append(btn)

# Create UI widgets for Tab 2 (Force-Stress Converter)
diameter_input = widgets.FloatText(
    value=10.0,
    min=0.1,
    max=1000.0,
    step=0.1,
    description='Diameter (mm):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

stress_input = widgets.FloatText(
    value=50.0,
    min=0.001,
    max=2500.0,
    step=5,
    description='Stress (MPa):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

force_output = widgets.FloatText(
    value=3.9,
    min=0.001,
    max=100.0,
    step=0.1,
    description='Force (kN):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px'),
    disabled=True  # Read-only output
)

# Buttons for Tab 2
calculate_force_button = widgets.Button(
    description='Calculate Force',
    button_style='success',
    icon='calculator',
    layout=widgets.Layout(width='200px')
)

clear_converter_button = widgets.Button(
    description='Clear All',
    button_style='warning',
    icon='eraser',
    layout=widgets.Layout(width='150px')
)

# Output area for Tab 2
converter_output = widgets.Output()
converter_results = widgets.HTML(value='')

# Create example buttons for Tab 2
converter_examples = [
    {'diameter': 10, 'stress': 50.0, 'label': '10mm, 70 MPa'},
    {'diameter': 10, 'stress': 500.0, 'label': '10mm, 500 MPa'},
    {'diameter': 15, 'stress': 50.0, 'label': '15 mm, 50MPa'},
    {'diameter': 15, 'stress': 70.0, 'label': '15mm, 70MPa'},
    {'diameter': 17, 'stress': 20.0, 'label': '17mm, 20MPa'},
    {'diameter': 17, 'stress': 50.0, 'label': '17mm, 50MPa'},
    {'diameter': 17, 'stress': 100.0, 'label': '17mm, 100MPa'}
]

converter_example_buttons = []
for example in converter_examples:
    btn = widgets.Button(
        description=example['label'],
        button_style='info',
        layout=widgets.Layout(width='150px', margin='0 5px 0 0')
    )
    btn.example = example
    converter_example_buttons.append(btn)

# Function to handle calculation for Tab 1
def calculate_formula(b):
    with result_output:
        result_output.clear_output()

        formula = input_formula.value.strip()
        if not formula:
            display(HTML('<div style="color: red; padding: 10px;">Please enter a chemical formula!</div>'))
            return

        try:
            # Parse formula
            elements = parse_chemical_formula(formula)

            if not elements:
                display(HTML(f'<div style="color: red; padding: 10px;">Could not parse formula: {formula}</div>'))
                return

            # Display what formula was interpreted as
            interpreted_formula = ''.join([f"{elem}{perc:.2f}" for elem, perc in elements])
            display(HTML(f'<div style="color: #666; padding: 5px; font-style: italic;">Interpreted as: {interpreted_formula}</div>'))

            # Calculate masses
            results = calculate_element_masses(elements)

            if not results:
                display(HTML(f'<div style="color: red; padding: 10px;">Error calculating masses for formula: {formula}</div>'))
                return

            # Display original and converted formulas
            original_formula = formula
            subscript_formula = create_formula_with_subscript(elements)
            superscript_formula = create_formula_with_percentage_superscript(elements)

            # Check if original formula had numbers
            has_numbers = any(char.isdigit() for char in original_formula)

            formula_display.value = f'''
            <div style="background-color: #f0f8ff; padding: 15px; border-radius: 5px; margin: 10px 0;">
                <h3 style="margin-top: 0;">Formula Analysis</h3>
                <table style="width: 100%; border-collapse: collapse;">
                    <tr>
                        <td style="padding: 5px; font-weight: bold;">Original:</td>
                        <td style="padding: 5px; font-family: monospace; font-size: 18px;">{original_formula}</td>
                    </tr>
                    <tr>
                        <td style="padding: 5px; font-weight: bold;">With Subscript:</td>
                        <td style="padding: 5px; font-family: monospace; font-size: 24px; color: #0066cc;">{subscript_formula}</td>
                    </tr>
                    <tr>
                        <td style="padding: 5px; font-weight: bold;">With % Superscript:</td>
                        <td style="padding: 5px; font-family: monospace; font-size: 24px; color: #cc3300;">{superscript_formula}</td>
                    </tr>
                </table>
                {"<div style='color: #ff6600; margin-top: 10px;'><strong>Note:</strong> No percentages were specified in the input, so 100% was divided equally among all elements.</div>" if not has_numbers else ""}
            </div>
            '''

            # Display element data table
            with mass_table_output:
                mass_table_output.clear_output()

                element_data = results['element_data']
                total_mass = results['total_mass']

                # Create DataFrame for display - XÓA CỘT "Mass (g/100g compound)"
                df = pd.DataFrame(element_data)

                # Xóa cột "Mass (g/100g compound)" nếu tồn tại
                if 'Mass (g/100g compound)' in df.columns:
                    df = df.drop(columns=['Mass (g/100g compound)'])

                # Format the DataFrame display
                display(HTML('<h3>Element Mass Calculation</h3>'))

                # Define formatting rules - ĐÃ XÓA CỘT "Mass (g/100g compound)"
                format_dict = {
                    'No.': '{:d}',
                    'Atomic %': '{:.2f}',
                    'Atomic Mass (g/mol)': '{:.4f}',
                    'Mass %': '{:.3f}',
                    'Mass in 10g (g)': '{:.4f}'
                }

                # Apply styling với 2 cột cuối cùng có header giống nhau và giá trị in đậm
                # Bây giờ chúng ta có 5 cột: No., Element, Atomic %, Atomic Mass (g/mol), Mass %, Mass in 10g (g)
                # 2 cột cuối là: Mass % và Mass in 10g (g)
                styled_df = df.style.format(format_dict).set_properties(**{
                    'text-align': 'center',
                    'border': '1px solid #ddd',
                    'padding': '5px'
                }).set_table_styles([
                    {'selector': 'thead th', 'props': [
                        ('background-color', '#4CAF50'),
                        ('color', 'white'),
                        ('font-weight', 'bold'),
                        ('text-align', 'center')
                    ]},
                    {'selector': 'tbody tr:nth-child(even)', 'props': [
                        ('background-color', '#f2f2f2')
                    ]},
                    {'selector': 'tbody tr:hover', 'props': [
                        ('background-color', '#ddd')
                    ]},
                    # STT column (first column)
                    {'selector': 'td:nth-child(1)', 'props': [
                        ('font-weight', 'bold'),
                        ('background-color', '#f8f9fa')
                    ]},
                    # 2 cột cuối: header giống nhau (cột 6, 7) - Mass % và Mass in 10g (g)
                    {'selector': 'th:nth-child(6), th:nth-child(7)', 'props': [
                        ('font-weight', 'bold'),
                        ('color', 'white'),
                        ('text-align', 'center')
                    ]},
                    # 2 cột cuối: giá trị in đậm, không có màu, căn phải
                    {'selector': 'td:nth-child(6), td:nth-child(7)', 'props': [
                        ('font-weight', 'bold'),
                        ('text-align', 'right')
                    ]}
                ])

                display(styled_df)

                # Display summary
                display(HTML(f'''
                <div style="background-color: #e8f5e8; padding: 15px; border-radius: 5px; margin-top: 15px;">
                    <h4 style="margin: 0 0 10px 0;">Summary</h4>
                    <table style="width: 100%; border-collapse: collapse;">
                        <tr>
                            <td style="padding: 5px; font-weight: bold;">Total mass per 100g compound:</td>
                            <td style="padding: 5px; text-align: right;"><strong>{total_mass:.4f} g</strong></td>
                        </tr>
                        <tr>
                            <td style="padding: 5px; font-weight: bold;">Total mass for 10g compound:</td>
                            <td style="padding: 5px; text-align: right;"><strong>10.0000 g</strong></td>
                        </tr>
                        <tr>
                            <td style="padding: 5px; font-weight: bold;">Number of elements:</td>
                            <td style="padding: 5px; text-align: right;"><strong>{len(elements)}</strong></td>
                        </tr>
                        <tr>
                            <td style="padding: 5px; font-weight: bold;">Total atomic % (should be 100%):</td>
                            <td style="padding: 5px; text-align: right;"><strong>{sum(item.get("Atomic %", 0) for item in element_data if isinstance(item.get("Atomic %"), (int, float))):.2f} %</strong></td>
                        </tr>
                        <tr>
                            <td style="padding: 5px; font-weight: bold;">Total mass % (should be 100%):</td>
                            <td style="padding: 5px; text-align: right;"><strong>{sum(item.get("Mass %", 0) for item in element_data if isinstance(item.get("Mass %"), (int, float))):.3f} %</strong></td>
                        </tr>
                    </table>
                </div>
                '''))

            # Display additional information
            with summary_output:
                summary_output.clear_output()

                # Show how to interpret results - Cập nhật không còn đề cập đến cột "Mass (g/100g compound)"
                display(HTML(f'''
                <div style="background-color: #fff3cd; padding: 15px; border-radius: 5px; margin-top: 15px; border: 1px solid #ffeaa7;">
                    <h4 style="margin-top: 0; color: #856404;">📊 Interpretation Guide</h4>
                    <ul style="margin-bottom: 10px;">
                        <li><strong>Atomic %</strong>: Percentage of atoms of each element in the compound (atomic basis)</li>
                        <li><strong>Atomic Mass</strong>: Mass of one mole of atoms (g/mol)</li>
                        <li><strong>Mass %</strong>: Percentage of total mass contributed by each element (mass basis)</li>
                        <li><strong>Mass in 10g (g)</strong>: Mass of each element in 10g of the compound (practical measurement)</li>
                    </ul>
                    <p style="margin: 0;"><strong>Correct Formulas:</strong></p>
                    <ul style="margin: 5px 0;">
                        <li>Mass % = (Atomic % × Atomic Mass) / Σ(Atomic % × Atomic Mass) × 100%</li>
                        <li>Mass in 10g = (Mass % / 100) × 10</li>
                    </ul>
                </div>

                <div style="background-color: #e3f2fd; padding: 15px; border-radius: 5px; margin-top: 15px; border: 1px solid #bbdefb;">
                    <h4 style="margin-top: 0; color: #1565c0;">💡 Input Options</h4>
                    <p style="margin: 0 0 10px 0;">
                        <strong>Two ways to input:</strong>
                    </p>
                    <ol>
                        <li>With percentages: <code>Al82Fe18</code> → Al: 82%, Fe: 18%</li>
                        <li>Without percentages: <code>AlNiCoTiFeMo</code> → Equal distribution (100% ÷ 6 = 16.67% each)</li>
                    </ol>
                </div>
                '''))

        except Exception as e:
            display(HTML(f'<div style="color: red; padding: 10px;">Error: {str(e)}</div>'))

# Function to handle clearing for Tab 1
def clear_all(b):
    input_formula.value = ''
    result_output.clear_output()
    formula_display.value = ''
    mass_table_output.clear_output()
    summary_output.clear_output()

# Function to handle example button clicks for Tab 1
def load_example(b):
    input_formula.value = b.formula
    calculate_formula(b)

# Function to handle calculation for Tab 2
def calculate_force_handler(_):
    with converter_output:
        converter_output.clear_output()
        try:
            d = float(diameter_input.value)
            s = float(stress_input.value)

            if d <= 0:
                display(HTML('<div style="color: red; padding: 10px;">Diameter must be greater than 0.</div>'))
                return
            if s < 0:
                display(HTML('<div style="color: red; padding: 10px;">Stress must be non-negative.</div>'))
                return

            force_kn, area_mm2 = calculate_force_from_stress(s, d)
            # Chuyển đổi kN sang tấn (ton): 1 kN = 0.101971621 ton
            force_ton = force_kn * 0.101971621
            # Cập nhật với định dạng 2 chữ số thập phân
            force_output.value = f"{force_kn:.2f}"

            converter_results.value = f"""
            <div style="background-color: #f8f0ff; padding: 15px; border-radius: 5px; margin: 10px 0;">
                <h3 style="margin-top: 0;">Calculation Results</h3>
                <table style="width: 100%; border-collapse: collapse;">
                    <tr><td style="padding: 5px 5px 5px 20px;">Diameter:</td><td style="padding: 5px;"><strong>{d:.2f} mm</strong></td></tr>
                    <tr><td style="padding: 5px 5px 5px 20px;">Stress:</td><td style="padding: 5px;"><strong>{s:.2f} MPa</strong></td></tr>
                    <tr><td style="padding: 5px 5px 5px 20px;">Area:</td><td style="padding: 5px;"><strong>{area_mm2:.4f} mm²</strong></td></tr>
                    <tr><td style="padding: 5px 5px 5px 20px;">Force:</td><td style="padding: 5px;"><strong style="color: #cc3300; font-size: 18px;">{force_kn:.2f} kN ({force_ton:.2f} ton)</strong></td></tr>
                </table>
                <div style="margin-top: 15px; padding: 10px; background-color: #e8f5e8; border-radius: 5px;">
                    <strong>Formula:</strong> F = σ × A / 1000<br>
                    <strong>Conversion:</strong> 1 kN = 0.101971621 ton
                </div>
            </div>
            """
        except Exception as e:
            display(HTML(f'<div style="color: red; padding: 10px;">Error: {str(e)}</div>'))

def clear_converter(b):
    diameter_input.value = 10.0
    stress_input.value = 12.732
    force_output.value = 1.0
    converter_results.value = ''
    with converter_output:
        converter_output.clear_output()

def load_converter_example(b):
    example = b.example
    diameter_input.value = example['diameter']
    stress_input.value = example['stress']

    # Auto-calculate force for the example
    calculate_force_handler(b)

# Function to auto-calculate when diameter or stress changes
def auto_calculate_force(change):
    # Calculate force automatically when diameter or stress changes
    if diameter_input.value > 0 and stress_input.value > 0:
        force_kn, _ = calculate_force_from_stress(stress_input.value, diameter_input.value)
        force_output.value = force_kn

# Set up automatic calculation when inputs change
diameter_input.observe(auto_calculate_force, names='value')
stress_input.observe(auto_calculate_force, names='value')

# Set up event handlers for Tab 1
calculate_button.on_click(calculate_formula)
clear_button.on_click(clear_all)

for btn in example_buttons:
    btn.on_click(load_example)

# Set up event handlers for Tab 2
calculate_force_button.on_click(calculate_force_handler)
clear_converter_button.on_click(clear_converter)

for btn in converter_example_buttons:
    btn.on_click(load_converter_example)

# Create Tab 1 content
tab1_content = widgets.VBox([
    widgets.HTML('<h2>🧪 Chemical Formula Calculator</h2>'),
    widgets.HTML('<p>Enter chemical formula with or without atomic percentages:</p>'),
    widgets.HTML('''<ul style="margin-top: 0; padding-left: 20px;">
                 <li>With percentages: <code>Al82Fe18</code></li>
                 <li>Without percentages (equal distribution): <code>AlNiCoTiFeMo</code></li>
                 </ul>'''),

    widgets.HBox([input_formula, calculate_button, clear_button]),

    widgets.HTML('<h3>Examples:</h3>'),
    widgets.HBox(example_buttons),

    widgets.HTML('<hr>'),

    formula_display,

    widgets.HTML('<h3>📈 Element Mass Calculation</h3>'),
    mass_table_output,

    summary_output,

    result_output
])

# Create Tab 2 content
tab2_content = widgets.VBox([
    widgets.HTML('<h2>🔧 Stress to Force Converter</h2>'),
    widgets.HTML('<p>Convert stress (MPa) to force (kN) for a circular cross-section:</p>'),

    widgets.HTML('<h3>Input Parameters</h3>'),
    diameter_input,
    stress_input,

    widgets.HTML('<hr>'),

    widgets.HTML('<h3>Calculated Force</h3>'),
    widgets.HTML('<p>Force will be automatically calculated when you change diameter or stress:</p>'),
    force_output,

    widgets.HBox([calculate_force_button, clear_converter_button]),

    widgets.HTML('<hr>'),

    widgets.HTML('<h3>Quick Examples:</h3>'),
    widgets.HBox(converter_example_buttons),

    widgets.HTML('<hr>'),

    converter_results,
    converter_output,

    widgets.HTML('''
    <div style="background-color: #fff3cd; padding: 15px; border-radius: 5px; margin-top: 15px; border: 1px solid #ffeaa7;">
        <h4 style="margin-top: 0; color: #856404;">📊 Formulas</h4>
        <ul style="margin-bottom: 10px;">
            <li><strong>Cross-sectional Area:</strong> A = π × (d/2)²</li>
            <li><strong>Force from Stress:</strong> F = σ × A / 1000 (kN)</li>
            <li><strong>Where:</strong> 1 kN = 1000 N, 1 MPa = 1 N/mm²</li>
        </ul>
        <p style="margin: 0;"><strong>Example:</strong> For diameter 10 mm and stress 12.732 MPa → Force = 1.000 kN</p>
    </div>
    ''')
])

# Create the tab widget
tab_widget = widgets.Tab()
tab_widget.children = [tab1_content, tab2_content]
tab_widget.set_title(0, '🧪 Chemical Formula')
tab_widget.set_title(1, '🔧 Stress→Force Converter')

# Display the UI
display(tab_widget)

# Calculate initial examples
calculate_formula(None)
calculate_force_handler(None)